In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


from scipy import stats
from sklearn import preprocessing

In [2]:
def convert_to_timedelta(df,date_col,td_col):
    DT = pd.to_datetime(df[date_col])
    min_date = DT.min()
    td_vals = DT.apply(lambda x:(x-min_date).days).values.astype(np.int)
    df[td_col] = td_vals - np.amin(td_vals)
    return(df)

def legacy_cumulation(col_vals,init_val=0,cumulation_type='mean'):
    legacy_sum = init_val
    legacy_count = 1
    legacy_cumulative_vals = []
    for i in col_vals:
        if(cumulation_type=='mean'):
            legacy_cumulative_vals.append(legacy_sum/legacy_count)
        else:
            legacy_cumulative_vals.append(legacy_sum)
        legacy_sum += i
        legacy_count += 1
    return(legacy_cumulative_vals)

def normalize_and_scale(df,col,offset=1):
    if((df[col].skew()>1.5)|(df[col].skew()<-1.5)):
        df[col] = df[col] + offset
        vals,_ = stats.boxcox(df[col].values)
        df[col] = vals.reshape(-1,1)
    df[col] = preprocessing.scale(df[col].values)
    return

def get_monotonocity(df,col):
    freq = dict(df[col].value_counts(normalize=True).sort_values(ascending=False))
    return(freq[df[col].mode().values[0]])

# Analyzing business features

## Business attributes

In [3]:
business_attr_file = "/home/pranav/ML/smart-interpreter/data_new.json"

In [4]:
business_attr = pd.read_json(business_attr_file,lines=True)
business_attr.head()

,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual,garage,...,halal,soy-free,vegetarian,BYOB,AgesAllowed,RestaurantsCounterService,DietaryRestrictions,Open24Hours,BestNights,AcceptsInsurance
0,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,False,False,False,True,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,False,False,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,False,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
business_attr.dropna(axis=1,thresh=int(0.2*business_attr.shape[0]),inplace=True)

In [6]:
for attr in business_attr.columns[business_attr.columns!="business_id"]:
    business_attr[attr].fillna("NA",inplace=True)
    dummies = pd.get_dummies(business_attr[attr])
    dum_cols = dummies.columns
    for col in dum_cols:
        if(get_monotonocity(dummies,col))>0.8:
            dummies.drop(col,axis=1,inplace=True)
    dummies.columns = [(attr+"_"+col) for col in dummies.columns]
    business_attr = pd.concat((business_attr.drop(attr,axis=1),dummies),axis=1)

business_attr.head()    

,business_id,divey_False,divey_NA,trendy_False,casual_False,casual_True,garage_False,street_False,lot_False,lot_True,...,dinner_NA,dinner_True,brunch_False,brunch_NA,breakfast_False,breakfast_NA,BikeParking_NA,BikeParking_True,RestaurantsTableService_NA,RestaurantsTableService_True
0,eU_713ec6fTGNO4BegRaww,1,0,1,1,0,1,1,0,1,...,0,1,1,0,1,0,0,1,1,0
1,sMzNLdhJZGzYirIWt-fMAg,1,0,0,0,1,1,0,1,0,...,0,1,1,0,1,0,0,1,1,0
2,xOG2NIFCsjBrVvWm69xkvw,1,0,1,1,0,1,0,1,0,...,1,0,0,1,0,1,0,1,1,0
3,oPvYovnl06fy5W0e51zjMQ,1,0,1,0,1,1,0,1,0,...,0,1,1,0,1,0,0,1,1,0
4,wXNHLkDkh0HKGJeYVnpafQ,1,0,1,0,1,1,0,1,0,...,0,1,1,0,1,0,0,1,1,0


In [7]:
# Create correlation matrix
corr_matrix = business_attr.corr(method='kendall').abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
business_attr.drop(to_drop,axis=1,inplace=True)

In [8]:
business_attr.head()

,business_id,divey_False,trendy_False,casual_False,casual_True,garage_False,street_False,lot_False,lot_True,NoiseLevel_NA,...,Caters_True,dessert_False,lunch_False,lunch_True,dinner_False,dinner_True,BikeParking_NA,BikeParking_True,RestaurantsTableService_NA,RestaurantsTableService_True
0,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,1,1,0,0,1,0,1,1,0
1,sMzNLdhJZGzYirIWt-fMAg,1,0,0,1,1,0,1,0,0,...,0,1,1,0,0,1,0,1,1,0
2,xOG2NIFCsjBrVvWm69xkvw,1,1,1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,1,0
3,oPvYovnl06fy5W0e51zjMQ,1,1,0,1,1,0,1,0,0,...,0,1,0,1,0,1,0,1,1,0
4,wXNHLkDkh0HKGJeYVnpafQ,1,1,0,1,1,0,1,0,0,...,0,1,0,1,0,1,0,1,1,0


In [9]:
business_df = pd.read_csv("/home/pranav/ML/smart-interpreter/Reviews.csv")[["business_id","review_id","date","stars_review","useful_review","funny_review","cool_review"]]
business_df.head()

,business_id,review_id,date,stars_review,useful_review,funny_review,cool_review
0,eU_713ec6fTGNO4BegRaww,Ci7eLn5l1X1UTIe0COaCrA,2015-06-08 18:51:54,5,1,0,0
1,sMzNLdhJZGzYirIWt-fMAg,BVDLUZbUUDDIHt8QD5zjSw,2015-12-08 16:04:39,4,0,0,0
2,xOG2NIFCsjBrVvWm69xkvw,b_2ksNbu_b9mOG_IPptoEQ,2016-01-18 21:20:07,2,3,0,0
3,oPvYovnl06fy5W0e51zjMQ,mtIaagwcd3YADZDahJYoYA,2016-02-22 15:55:00,3,0,0,0
4,wXNHLkDkh0HKGJeYVnpafQ,0qBFP_RtitEMsYZ6rnAzqQ,2016-01-14 20:25:45,5,0,0,0


In [10]:
convert_to_timedelta(business_df,"date","date_td")
business_df.head()

,business_id,review_id,date,stars_review,useful_review,funny_review,cool_review,date_td
0,eU_713ec6fTGNO4BegRaww,Ci7eLn5l1X1UTIe0COaCrA,2015-06-08 18:51:54,5,1,0,0,3748
1,sMzNLdhJZGzYirIWt-fMAg,BVDLUZbUUDDIHt8QD5zjSw,2015-12-08 16:04:39,4,0,0,0,3931
2,xOG2NIFCsjBrVvWm69xkvw,b_2ksNbu_b9mOG_IPptoEQ,2016-01-18 21:20:07,2,3,0,0,3973
3,oPvYovnl06fy5W0e51zjMQ,mtIaagwcd3YADZDahJYoYA,2016-02-22 15:55:00,3,0,0,0,4007
4,wXNHLkDkh0HKGJeYVnpafQ,0qBFP_RtitEMsYZ6rnAzqQ,2016-01-14 20:25:45,5,0,0,0,3968


In [11]:
business_df.sort_values(by=["business_id","date_td"],inplace=True)

In [12]:
init_val_stars = business_df["stars_review"].mean()
business_df["business_legacy_stars_mean"] = business_df.groupby('business_id')["stars_review"].transform(lambda x:legacy_cumulation(x,init_val_stars,'mean')[:len(x)]).reset_index()["stars_review"].values
business_df["business_legacy_funny"] = business_df.groupby('business_id')["funny_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["funny_review"].values
business_df["business_legacy_cool"] = business_df.groupby('business_id')["cool_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["cool_review"].values
business_df["business_legacy_useful"] = business_df.groupby('business_id')["useful_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["useful_review"].values

In [13]:
business_df.head(n=10)

,business_id,review_id,date,stars_review,useful_review,funny_review,cool_review,date_td,business_legacy_stars_mean,business_legacy_funny,business_legacy_cool,business_legacy_useful
92994,--GM_ORV2cYS-h38DSaCLw,grm_MOzOKIw-icfJXv0-KQ,2017-01-16 09:45:52,4,0,0,1,4336,3.761209,0,0,0
517,--GM_ORV2cYS-h38DSaCLw,yYgGRPPUVD863NlLz1Ry9g,2018-06-19 16:13:41,4,0,0,0,4855,3.880605,0,1,0
71210,--GM_ORV2cYS-h38DSaCLw,LE_PPqKTmt3TDrU7L3sprA,2018-08-01 08:10:10,2,0,0,0,4898,3.920403,0,1,0
133657,--KCl2FvVQpvjzmZSPyviA,j0OZX2IbCAu8uUSiyZLi9Q,2012-10-22 13:18:17,4,3,2,3,2789,3.761209,0,0,0
133658,--KCl2FvVQpvjzmZSPyviA,B1sHjhkqrtXYPoz_fCwuhg,2014-12-29 19:41:26,1,3,0,1,3587,3.880605,2,3,3
119293,--KCl2FvVQpvjzmZSPyviA,6O9WHWmJPzxcxmV3x9Ucbw,2015-10-16 22:29:45,2,0,0,0,3879,2.920403,2,4,6
119292,--KCl2FvVQpvjzmZSPyviA,KiPcHrzGOVKok8juy4lNog,2015-11-02 18:25:58,3,1,0,0,3895,2.690302,2,4,6
150735,--KCl2FvVQpvjzmZSPyviA,vArxmQcaflKP-PSsaTgZ6Q,2016-09-16 02:41:58,4,0,0,0,4214,2.752242,2,4,7
20345,-01XupAWZEXbdNbxNg5mEg,E-ETVFBYMy3tUfCrvJv73A,2009-09-30 06:54:34,4,6,6,5,1671,3.761209,0,0,0
34552,-01XupAWZEXbdNbxNg5mEg,NwiD6qcWyzcaCPPu3UJk9w,2009-10-25 03:36:49,4,5,2,5,1696,3.880605,6,5,6


In [14]:
business_df.drop(["stars_review","useful_review","funny_review","cool_review","date","date_td"],axis=1,inplace=True)

In [15]:
business_features = pd.merge(business_attr,business_df,on=["business_id"],how="right")
business_features.head()

,business_id,divey_False,trendy_False,casual_False,casual_True,garage_False,street_False,lot_False,lot_True,NoiseLevel_NA,...,dinner_True,BikeParking_NA,BikeParking_True,RestaurantsTableService_NA,RestaurantsTableService_True,review_id,business_legacy_stars_mean,business_legacy_funny,business_legacy_cool,business_legacy_useful
0,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,0,1,1,0,S1aROCxDQMyG_EYgpB-CAA,3.761209,0,0,0
1,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,0,1,1,0,0TDA4X4HK9PjZSz4TM5n5A,4.380605,2,3,4
2,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,0,1,1,0,M0Y9h_qdU_LF5_TkltRkJQ,4.587070,2,3,6
3,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,0,1,1,0,D2oYk9ipsvsYP_ls0aL9kg,4.690302,2,3,8
4,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,0,1,1,0,4yLl21v8cJEgJSjYCKoV5Q,4.552242,2,4,8


# User features

## User specific attributes

In [16]:
task1_file = "/home/pranav/ML/smart-interpreter/Reviews.csv"

In [17]:
user_cols = ["user_id","compliment_hot","compliment_more","compliment_profile","compliment_cute","compliment_list",
             "compliment_note","compliment_plain","compliment_cool","compliment_funny","compliment_writer",
             "compliment_photos","yelping_since"]
user_attr = pd.read_csv(task1_file)[user_cols].drop_duplicates(subset="user_id")
user_attr = user_attr.reset_index(drop=True)
user_attr.head()

,user_id,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,yelping_since
0,y0X59ZwPp89jVkceROxTtA,0,0,0,0,0,0,0,0,0,0,0,2014-03-11 13:36:24
1,9IA3P3DxrZcoAl29IYOloA,1,4,1,0,0,1,11,8,8,3,0,2009-03-16 18:17:31
2,Jk0AcqPCVMfc_DTWO3SzwA,3,0,0,0,0,4,5,1,1,7,1,2014-11-19 07:57:44
3,UsmTxWbobLsI6WR1Db0W7A,29,21,8,1,3,150,269,123,123,55,15,2010-08-09 06:58:27
4,GFR5vWJ0KVxuOIxlQFK-Lw,3,0,0,0,0,2,2,1,1,1,0,2012-03-20 16:34:06


In [18]:
convert_to_timedelta(user_attr,"yelping_since","yelping_since_td")
user_attr.drop("yelping_since",axis=1,inplace=True)
user_attr.head()

,user_id,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,yelping_since_td
0,y0X59ZwPp89jVkceROxTtA,0,0,0,0,0,0,0,0,0,0,0,3436
1,9IA3P3DxrZcoAl29IYOloA,1,4,1,0,0,1,11,8,8,3,0,1616
2,Jk0AcqPCVMfc_DTWO3SzwA,3,0,0,0,0,4,5,1,1,7,1,3689
3,UsmTxWbobLsI6WR1Db0W7A,29,21,8,1,3,150,269,123,123,55,15,2126
4,GFR5vWJ0KVxuOIxlQFK-Lw,3,0,0,0,0,2,2,1,1,1,0,2715


In [19]:
# for attr in user_attr.columns[user_attr.columns!="user_id"]:
#     if((user_attr[attr].skew()>1.5)|(user_attr[attr].skew()<-1.5)):
#         print(attr+" is highly skewed.")
#         normalize_and_scale(user_attr,attr,1)
#     else:
#         scale(user_attr,attr,1)
# user_attr.head()

In [20]:
user_df = pd.read_csv("/home/pranav/ML/smart-interpreter/Reviews.csv")[["user_id","review_id","date","stars_review","useful_review","funny_review","cool_review"]]
user_df.head()

,user_id,review_id,date,stars_review,useful_review,funny_review,cool_review
0,y0X59ZwPp89jVkceROxTtA,Ci7eLn5l1X1UTIe0COaCrA,2015-06-08 18:51:54,5,1,0,0
1,y0X59ZwPp89jVkceROxTtA,BVDLUZbUUDDIHt8QD5zjSw,2015-12-08 16:04:39,4,0,0,0
2,y0X59ZwPp89jVkceROxTtA,b_2ksNbu_b9mOG_IPptoEQ,2016-01-18 21:20:07,2,3,0,0
3,y0X59ZwPp89jVkceROxTtA,mtIaagwcd3YADZDahJYoYA,2016-02-22 15:55:00,3,0,0,0
4,y0X59ZwPp89jVkceROxTtA,0qBFP_RtitEMsYZ6rnAzqQ,2016-01-14 20:25:45,5,0,0,0


In [21]:
convert_to_timedelta(user_df,"date","date_td")
user_df.head()

,user_id,review_id,date,stars_review,useful_review,funny_review,cool_review,date_td
0,y0X59ZwPp89jVkceROxTtA,Ci7eLn5l1X1UTIe0COaCrA,2015-06-08 18:51:54,5,1,0,0,3748
1,y0X59ZwPp89jVkceROxTtA,BVDLUZbUUDDIHt8QD5zjSw,2015-12-08 16:04:39,4,0,0,0,3931
2,y0X59ZwPp89jVkceROxTtA,b_2ksNbu_b9mOG_IPptoEQ,2016-01-18 21:20:07,2,3,0,0,3973
3,y0X59ZwPp89jVkceROxTtA,mtIaagwcd3YADZDahJYoYA,2016-02-22 15:55:00,3,0,0,0,4007
4,y0X59ZwPp89jVkceROxTtA,0qBFP_RtitEMsYZ6rnAzqQ,2016-01-14 20:25:45,5,0,0,0,3968


In [22]:
user_df.sort_values(by=["user_id","date_td"],inplace=True)

In [23]:
init_val_stars = user_df["stars_review"].mean()
user_df["user_legacy_stars_mean"] = user_df.groupby('user_id')["stars_review"].transform(lambda x:legacy_cumulation(x,init_val_stars,'mean')[:len(x)]).reset_index()["stars_review"].values
user_df["user_legacy_funny"] = user_df.groupby('user_id')["funny_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["funny_review"].values
user_df["user_legacy_cool"] = user_df.groupby('user_id')["cool_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["cool_review"].values
user_df["user_legacy_useful"] = user_df.groupby('user_id')["useful_review"].transform(lambda x:legacy_cumulation(x,0,'sum')[:len(x)]).reset_index()["useful_review"].values

In [24]:
user_df.head()

,user_id,review_id,date,stars_review,useful_review,funny_review,cool_review,date_td,user_legacy_stars_mean,user_legacy_funny,user_legacy_cool,user_legacy_useful
79639,---1lKK3aKOuomHnwAkAow,nHYLl06G_Yt8dcRpzCJFiQ,2010-11-05 20:08:08,1,2,1,0,2072,3.761209,0,0,0
79643,---1lKK3aKOuomHnwAkAow,7EQzYGniK8TJvEOkMaTDyg,2010-11-16 03:11:16,5,2,0,2,2083,2.380605,1,0,2
79641,---1lKK3aKOuomHnwAkAow,embcko9m0u_e6z7w3ki2TA,2010-11-25 05:48:23,5,2,0,1,2092,3.253736,1,2,4
79645,---1lKK3aKOuomHnwAkAow,iE3bwhCjGCLBcKUGN7jjkQ,2010-11-25 06:37:32,5,3,1,1,2092,3.690302,1,3,6
79635,---1lKK3aKOuomHnwAkAow,zPKdPn9gALWT2BVOTntj3g,2011-02-01 20:52:31,5,1,0,1,2161,3.952242,2,4,9


In [25]:
user_df.drop(["date","stars_review","useful_review","funny_review","cool_review","date_td"],axis=1,inplace=True)
user_df.head()

,user_id,review_id,user_legacy_stars_mean,user_legacy_funny,user_legacy_cool,user_legacy_useful
79639,---1lKK3aKOuomHnwAkAow,nHYLl06G_Yt8dcRpzCJFiQ,3.761209,0,0,0
79643,---1lKK3aKOuomHnwAkAow,7EQzYGniK8TJvEOkMaTDyg,2.380605,1,0,2
79641,---1lKK3aKOuomHnwAkAow,embcko9m0u_e6z7w3ki2TA,3.253736,1,2,4
79645,---1lKK3aKOuomHnwAkAow,iE3bwhCjGCLBcKUGN7jjkQ,3.690302,1,3,6
79635,---1lKK3aKOuomHnwAkAow,zPKdPn9gALWT2BVOTntj3g,3.952242,2,4,9


In [26]:
user_features = pd.merge(user_attr,user_df,on=["user_id"],how="right")

# Merging, scaling features and writing to file

In [27]:
total_data = pd.merge(business_features,user_features,on=["review_id"],how="outer")
total_data_target = pd.read_csv(task1_file)[["review_id","stars_review"]]
total_data = pd.merge(total_data,total_data_target,on="review_id",how="outer")
print(total_data.shape)
total_data.head()

(163528, 61)


,business_id,divey_False,trendy_False,casual_False,casual_True,garage_False,street_False,lot_False,lot_True,NoiseLevel_NA,...,compliment_cool,compliment_funny,compliment_writer,compliment_photos,yelping_since_td,user_legacy_stars_mean,user_legacy_funny,user_legacy_cool,user_legacy_useful,stars_review
0,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,277,277,114,4,1065,3.440302,1,5,9,5
1,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1,1,1,0,1656,3.761209,0,0,0,5
2,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,0,0,1,0,1703,3.587070,0,0,1,5
3,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,8,8,3,0,1616,4.253736,0,0,5,4
4,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,6,6,4,3,1874,4.253736,0,0,5,5


In [28]:
#Normalizing skewed continuous variables and scaling between 0 and 1
key_columns = ["review_id","business_id","user_id","stars_review"]
cols = [col for col in total_data.columns if col not in key_columns]
for col in cols:
    if(total_data[col].max()>1):
        normalize_and_scale(total_data,col,offset=1)
total_data.head()

,business_id,divey_False,trendy_False,casual_False,casual_True,garage_False,street_False,lot_False,lot_True,NoiseLevel_NA,...,compliment_cool,compliment_funny,compliment_writer,compliment_photos,yelping_since_td,user_legacy_stars_mean,user_legacy_funny,user_legacy_cool,user_legacy_useful,stars_review
0,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,1.612997,1.612997,1.622248,0.880530,-1.781205,-0.612016,-0.011532,0.784651,0.561200,5
1,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,-0.598520,-0.598520,-0.461657,-0.905951,-1.119404,0.006065,-1.012013,-1.094688,-1.375735,5
2,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,-1.206495,-1.206495,-0.461657,-0.905951,-1.066774,-0.329335,-1.012013,-1.094688,-0.683897,5
3,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,0.395105,0.395105,0.106314,-0.905951,-1.164196,0.954695,-1.012013,-1.094688,0.213703,4
4,eU_713ec6fTGNO4BegRaww,1,1,1,0,1,1,0,1,0,...,0.255156,0.255156,0.264205,0.724110,-0.875288,0.954695,-1.012013,-1.094688,0.213703,5


In [29]:
total_data.to_csv("task1_processed_data.csv",index=False)